In [1]:
%%capture

!pip install elasticsearch==7.14.0
!apt install default-jdk > /dev/null

In [2]:
try:
  import os
  import elasticsearch
  from elasticsearch import Elasticsearch
  import numpy as np
  import pandas as pd
  import sys
  import json
  from ast import literal_eval
  from tqdm import tqdm
  import datetime
  from elasticsearch import helpers

except Exception as e:
  print(f"error: {e}")

In [3]:
!wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.0.0-linux-x86_64.tar.gz -q
!tar -xzf elasticsearch-7.0.0-linux-x86_64.tar.gz
!chown -R daemon:daemon elasticsearch-7.0.0

In [4]:
import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.0.0/bin/elasticsearch'],
                  stdout=PIPE, stderr=STDOUT,
                  preexec_fn=lambda: os.setuid(1)  # as daemon
                 )

In [5]:
import time
time.sleep(20)

In [6]:
%%bash
# If you get 1 root & 2 daemon process then Elasticsearch instance has started successfully
ps -ef | grep elasticsearch

daemon       970     462 99 09:46 ?        00:00:25 /content/elasticsearch-7.0.0/jdk/bin/java -Xms1g
daemon      1074     970  0 09:46 ?        00:00:00 /content/elasticsearch-7.0.0/modules/x-pack-ml/p


In [7]:
# 1
!curl -sX GET "localhost:9200/"

In [ ]:
es = Elasticsearch(hosts = [{"host":"localhost", "port":9200}])
# Check if python is connected to elasticsearch
es.ping()

In [10]:
# Elasticsearch 인덱스 생성
index_name = 'txt_data_index'
if not es.indices.exists(index=index_name):
    es.indices.create(index=index_name)

In [11]:
#구글드라이브 마운트하기
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install sentence-transformers

In [ ]:
!pip install langchain unstructured openai chromadb Cython tiktoken pypdf

In [14]:
from langchain.text_splitter import CharacterTextSplitter

In [15]:
# 폴더에서 데이터 읽어오기
folder_path = "/content/drive/My Drive/Colab Notebooks/data/school_text"

# 폴더 내의 모든 파일 목록 가져오기
file_list = os.listdir(folder_path)

# text_splitter 초기화
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=50)

# 각 텍스트 파일을 읽어와서 Elasticsearch에 색인화
for i, file_name in enumerate(file_list):
    file_path = os.path.join(folder_path, file_name)
    with open(file_path, 'r', encoding='utf-8') as file:
        data = file.read()

    # text_splitter를 사용하여 텍스트 분리
    chunks = text_splitter.split_text(data)

    # 데이터를 Elasticsearch에 색인화
    for j, chunk in enumerate(chunks):
        document = {
            'text': chunk.strip(),  # 각 줄의 텍스트를 'text' 필드에 저장
            'file_name': file_name  # 파일 이름도 저장 (옵션)
        }
        es.index(index=index_name, doc_type='_doc', id=i * len(file_list) + j + 1, body=document)


In [16]:
# 색인화가 완료되었다면 몇 초 기다려서 Elasticsearch에 적용되도록 합시다.
import time
time.sleep(5)

In [81]:
# 특정 쿼리로 검색하기
search_query = '장학생 선발 절차는?' # 여기 입력
search_body = {
    'query': {
        'match': {
            'text': search_query
        }
    },
    'size': 5 # 최대 5개의 문서만 반환(토큰 4000제한 때문에)
}

In [82]:
# 검색 실행
results = es.search(index=index_name, body=search_body)

In [83]:
# 결과 출력
print(f"검색어 '{search_query}'에 대한 검색 결과:")
for hit in results['hits']['hits']:
    print(hit['_source']['text'])

# top_hit = results['hits']['hits'][0]['_source']['text']
# print(top_hit)

검색어 '장학생 선발 절차는?'에 대한 검색 결과:
나. 장학생 선정 대상자는 직전학기에 15학점이상 이수하여야 한다.(직전학기 성적에 계절학기 및 국내․외 타대학교류 학점인정 성적은 포함하지 않는다) 다만, 다음의 경우에는 예외로 한다.

      * 직전학기가 4학년 1학기인 경우는 8학점(건축학은 5학년 1학기) 이상

  다. 공인외국어시험성적은 [별표 1] “공인외국어시험 점수 환산표”의 토익 최저점수(300점)까지만 인정한다.

3. 장학생 선발

  가. 외국어성적우수장학생은 학기별 50명 내외로 공인외국어성적이 우수한 순으로 선발한다.
등록금 20%

 직전 2개 학기 성적이 평점평균 3.0 이상  

금오, 우등과 동일

※ (공통기준) 이수학점 : 직전학기 15학점 이상[단, 직전학기가 4학년(건축학전공은 5학년)인 경우  8학점 이상]

※ 위 기준은 기본자격기준이며, 

   금오, 우등은 기준 충족자 중 학부(과)에서 장학생 추천 → 최종 선발

   성적오름은 상승편차가 큰 자 중 최종 선발

▼

氠瑢    

“학생” 외국어성적 등록 및 승인요청
5. 제출점수 입력방법 : ‘원스톱서비스-학생지원-교내장학금-외국어성적우수 제출점수입력’ 메뉴에서 기간내 접수, 승인된 공인외국어성적을 선택하여 저장

   ※ 제출점수 입력방법에 관한 자세한 사항은 별도 붙임파일을 꼭 확인하신 후 입력바람

6. 장학생 선발 방법

  가. 다음과 같은 평가 점수가 우수한자 순

      평가 점수 = 100×(제출점수-기준점수)/(1100-제출점수)+(제출점수-기준점수)/5

  나. 학부 및 전공과 무관하게 평가점수 우수자 순으로 선발
捤獥    汤捯    湰灧    외국어성적우수장학금 지급 기준

1. 자격기준

   氠瑢    

장학금종류

대상 및 자격기준

제출서류

장학금액

외국어성적

우수장학금

◦공인외국어성적이 600점(토익기준) 이상인 자

◦직전학기 성적이 평점평균 2.6이상

공인

외국어시험 성적표

40

#gpt 연결

In [ ]:
!pip install --upgrade pip

In [ ]:
!pip install --upgrade llmx

In [ ]:
!pip install cohere

In [ ]:
!pip install llmx==0.0.15a0 cohere==<버전>

In [ ]:
!pip install openai==0.28

In [48]:
import openai

openai.api_key = '<Your_OpenAI_APIkey>'

In [59]:
# GPT를 사용하여 쿼리를 생성하는 함수
def generate_query(prompt):
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        max_tokens=500 
    )
    return response.choices[0].text.strip()

In [85]:
# 결과가 있을 경우 결과를 GPT에 입력하여 응답 생성
if results.get('hits', {}).get('total', {}).get('value', 0) != 0:
    # 개행 문자 제거 후 최대 길이까지 자르기
    formatted_texts = [hit['_source']['text'].replace('\n', ', ') for hit in results['hits']['hits']]
    summary_prompt = '\n'.join(formatted_texts)

    # 리스트를 문자열로 조합
    formatted_result = ' '.join(formatted_texts)
    # GPT를 사용하여 쿼리를 생성합니다
    gpt_summary_query = generate_query(f"{summary_prompt} 중에서 '{search_query}'에 대한 답을 찾아줘.")

    # GPT로 생성된 요약 출력
    print("GPT로 생성된 답변:", gpt_summary_query)
else:
    print(f"검색어 '{search_query}'에 대한 검색 결과가 없습니다.")

GPT로 생성된 답변: 장학생 선발 절차는 다음과 같다. 
1. 자격기준 및 지원 자격: 공인외국어성적, 직전학기 성적, 기본자격 등
2. 제출서류 제출: 공인외국어시험 성적증명서, 직전학기 성적증명서 등
3. 등록금 및 장학금 지급: 등록금 20% 지원 및 장학금 우수자 선발
4. 장학생 선발: 기준 충족자 중 학부(과)에서 장학생 추천 → 최종 선발


#스냅샷 생성(임시)

In [ ]:
from elasticsearch import Elasticsearch

# Elasticsearch 연결 설정
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

# 백업할 인덱스 이름과 스냅샷 이름 정의
index_name = 'your_index_name'
snapshot_name = 'snapshot_1'

# 스냅샷 생성
def create_snapshot(index, snapshot_name):
    es.snapshot.create_repository(repository='my_backup', body={'type': 'fs', 'settings': {'location': '/path/to/your/snapshot/directory'}})
    es.snapshot.create(repository='my_backup', snapshot=snapshot_name, body={'indices': index})

# 스냅샷 복원
def restore_snapshot(snapshot_name, index):
    es.indices.close(index=index)
    es.indices.clear_cache(index=index)
    es.snapshot.restore(repository='my_backup', snapshot=snapshot_name, body={'indices': index})
    es.indices.open(index=index)

# 스냅샷 생성
create_snapshot(index_name, snapshot_name)

# 스냅샷 복원 (예: 다른 인덱스로 복원)
restored_index_name = 'restored_index'
restore_snapshot(snapshot_name, restored_index_name)


#스냅샷 복원(임시)

일반적으로 Elasticsearch 스냅샷은 로컬 파일 시스템이나 원격 파일 시스템에 저장됩니다. 스냅샷 저장 위치는 Elasticsearch 설정 파일 (elasticsearch.yml)에서 지정하게 됩니다.

In [ ]:
from elasticsearch import Elasticsearch

# Elasticsearch 연결 설정
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

# 복원할 인덱스 이름 정의
restored_index_name = 'restored_index'

# 스냅샷 복원
def restore_snapshot(snapshot_name, index):
    es.indices.close(index=index)
    es.indices.clear_cache(index=index)
    es.snapshot.restore(repository='my_backup', snapshot=snapshot_name, body={'indices': index})
    es.indices.open(index=index)

# 스냅샷 복원
restore_snapshot('snapshot_1', restored_index_name)

# 복원된 인덱스에 대해 쿼리 수행
search_query = 'your_search_query'
search_body = {
    'query': {
        'match': {
            'text': search_query
        }
    }
}

# 쿼리 실행
results = es.search(index=restored_index_name, body=search_body)

# 결과 출력
print(f"검색어 '{search_query}'에 대한 검색 결과:")
for hit in results['hits']['hits']:
    print(f"파일: {hit['_source']['file_name']}, 텍스트: {hit['_source']['text']}")


#서버에서(임시)

Elasticsearch 서버 시작:
Elasticsearch 서버를 시작하여 새로운 설정이 적용되도록 합니다.

Linux에서는 다음과 같이 실행합니다:

bash
Copy code
sudo service elasticsearch start
Windows에서는 Elasticsearch가 설치된 디렉토리에서 bin/elasticsearch.bat 파일을 실행합니다.

로그 확인:
Elasticsearch 서버를 시작한 후 로그를 확인하여 어떤 오류도 없는지 확인하세요. 로그는 주로 {Elasticsearch 설치 디렉토리}/logs에 있습니다.

참고: Elasticsearch의 설정은 다양한 측면에서 수정될 수 있으며, 각 설정의 역할에 대해 이해하는 것이 중요합니다. 변경 사항이 Elasticsearch 클러스터에 영향을 미칠 수 있으므로 주의 깊게 수정해야 합니다. 또한, Elasticsearch의 버전에 따라 설정 항목이 다를 수 있으므로 해당 버전의 공식 문서를 참고하는 것이 좋습니다.